## Tabla de contenidos:
* [Descarga de datos](#descarga-datos)
* [Carga inicial de los datos](#carga-inicial)
* [Observación de valores nulos](#datos-nulos)
    * [Identificadores](#columnas-identificadores)
    * [Información](#columnas-informacion)
    * [Usuarios](#columnas-usuarios)
    * [Autores](#columnas-autores)
* [Distribución y Correlaciones](#distribucion-correlacion)

## Descarga de datos <a class="anchor" name="descarga-datos"/>
Para la descarga de datos, se ha optado por el uso de un API.
En concreto se va a utilizar el API de goodreads, una página especializada en libros, con más de 85 millones de usuarios y más de 2.5 billones (americanos) de libros. ([https://www.goodreads.com/about/us](https://www.goodreads.com/about/us)).

El API ([https://www.goodreads.com/api](https://www.goodreads.com/api)) disponde un endpoint REST en el que se puede obtener la información detallada de un libro por _ISBN_ o por un _ID_ propio de su sistema. 
Para el caso de esta práctica se ha hecho un script en python (<a href="goodreads.py">*goodreads.py*</a>) que permite descargar la lista completa de libros en un fichero csv. 

Para facilitar el uso del notebook, se proporciorna un fichero csv con los libros descargados previamente: _books.csv_ 

Para usar el script bastaría con lanzar el comando: `python3 goodreads.py` 

Este script acepta parametros de entrada, como desde que libro hasta que libro se quiere ir, y cuantos libros se quieren saltar entre cada libro. `python3 goodreads.py -o FILENAME -s START_BOOK -e END_BOOK -l SKIP_BOOKS` 

## Carga inicial de los datos <a class="anchor" name="carga-inicial"/>

Se carga en memoria mediante pandas el dataset de libros almancenado en _books.csv_. 

Adicionalmente se obtiene la primera información sobre los datos que contine el dataset. 

In [ ]:
import pandas as pd

df = pd.read_csv("books.csv")
df.info()

In [ ]:
df.head()

## Observación de valores nulos <a class="anchor" name="datos-nulos"/>
El siguiente paso es observar los datos, los valores que toman y si hay valores nulos. 

Para cada una de las columnas del dataset se identifica si tienen valores nulos y cuántos hay en cada uno y la acción a tomar en cada caso.


Con los tipos y la muestra, se puede deducir la información de cada columna:

| Columna | Tipo | Observaciones | Tipo de Campo |
| --- | --- | --- | --- |
| id | int64 | ID en GoodReads | Identificador |
| isbn | object |  ISBN | Identificador |
| title | object |  Titulo | Información |
| isbn13 | float64 | Codigo ISBN de 13 digitos |  Identificador |
| asin | object | Identificador del libro en Amazon |  Identificador |
| kindle_asin | float64 | Codigo Kindle |  Identificador |
| marketplace_id | float64 | Código del mercado |  Identificador |
| country_code | object |  Código de País | Información |
| publication_date | object |  Fecha de publicación | Información | 
| publisher | object | Publicador/Editorial | Información |
| language_code | object | Código de idioma de la publicación | Información |
| is_ebook | bool | Es un E-Book | Información |
| books_count | int64 | ? | Usuarios |
| best_book_id | int64 | ? | Usuarios |
| reviews_count | int64 | Número de opiniones | Usuarios |
| ratings_sum | int64 | Sumatorio de las puntuaciones | Usuarios |
| ratings_count | int64 | Número de puntuaciones | Usuarios |
| text_reviews_count | int64 | Número de opiniones escritas | Usuarios |
| original_publication_date | object | Fecha de publicación original | Informacion |
| original_title | object| Titulo original | Informacion |
| media_type | object | Tipo de medio | Información |
| num_ratings_5 | int64 | Número de veces que han puntuado con un 5 | Usuarios |
| num_ratings_4 | int64 | Número de veces que han puntuado con un 4 | Usuarios |
| num_ratings_3 | int64 | Número de veces que han puntuado con un 3 | Usuarios |
| num_ratings_2 | int64 | Número de veces que han puntuado con un 2 | Usuarios |
| num_ratings_1 | int64 | Número de veces que han puntuado con un 1 | Usuarios |
| average_rating | float64 | Valoración media | Usuarios |
| num_pages | float64 | Número de páginas | Información |
| format | object | Formato | Información |
| edition_information | object  | Informacíón sobre la edición | Información |
| ratings_count_global | int64  | Número de valoraciones total | Usuarios |
| text_reviews_count_global | int64 | Número de reseñas total | Usuarios |
| authors | object | Autor | Autores |
| illustrator | object | Ilustrador | Autores | 
| contributor | object | Constribuidor | Autores |
| editor | object | Editor | Autores |
| translator | object | Traductor | Autores |
| narrator | object | Narrador | Autores |
| to_read | float64 | Número de personas que lo quieren leer | Usuarios |
| read | int64 | Número de personas que lo han leído | Usuarios |
| currently_reading | float64 | Número de personas que lo están leyendo | Usuarios |
| genres | object | Lista de géneros | Información |

Adicionalmente se ha identificado a que hace referencia cada columna:
* Identificador : Identificadores del libro
* Información: Información sobre el libro o la edición
* Usuarios: Información sobre las valoraciones de los usuarios, reseñas, etc.
* Autores: Información sobre los creadores del libro

Se observa cuantos valores nulos contiene cada columna

In [ ]:
df.isna().sum()

### Información sobre los identificadores <a class="anchor" name="columnas-identificadores"/>

Se eliminarán todas las filas que no contengan **isbn** ni **isbn13** dado que es un identificador de libro y va ayudar a eliminar libros que no tienen todos los datos cargados.

In [ ]:
df.dropna(subset=["isbn", "isbn13"], inplace=True)

Las columnas **asin**, **kindel_asin** y **marketplace_id** apenas si tienen valores, por lo que no parecen que vayan a ser estadísticamente relevantes por lo que se pueden eliminar del dataset.

In [ ]:
df.drop(columns = ['asin', 'kindle_asin', 'marketplace_id'], inplace=True)

### Información sobre el libro <a class="anchor" name="columnas-informacion"/>

Se observa que todos los libros tienen titulo(**title**) y practicamente todos tienes **original_title**. Se suprime los libros que no tienen alguno de esos dos campos debido al gran volumen de información que tenemos y así poder mezclar mejor los datos.

In [ ]:
df.dropna(subset=["title", "original_title"], inplace=True)

Las columna **country_code** no contiene nulos pero se ve que todos los valores son iguales por lo que podría ser eliminada.

In [ ]:
df.country_code.unique()

In [ ]:
df.drop(columns = ['country_code'], inplace=True)

Debido a que hay pocos libros con fecha original de publicación (*original_publication_date*) y se considera que es un campo importante sobre el que hacer muestreo y obtener información, se toma la decisión de borrar dichos libros.

In [ ]:
print("publication_date", df.publication_date.isna().sum())
print("original_publication_date", df.original_publication_date.isna().sum())
df.dropna(subset=["original_publication_date"], inplace=True)
print("publication_date", df.publication_date.isna().sum())
print("original_publication_date", df.original_publication_date.isna().sum())

Tanto la columna **publisher** como **language_code** contienen nulos y deben ser tratados como *unkown* en lugar de como nulos, dado que es una información que no ha sido proporcionada.

In [ ]:
df.publisher.fillna(value="unknown", inplace=True)
df.language_code.fillna(value="unknown", inplace=True)

La columna **is_ebook** no contine nulos y tiene un valor boolean en el que indica si un libro está en formato electrónico o no.

In [ ]:
df.is_ebook.unique()

Parece que la columna **media_type** tiene nulos, se obtienen los valores únicos:

In [ ]:
df["media_type"].unique()

Se comprueba qué tipo de elementos son considerados *not a book*:

In [ ]:
df[["title", "media_type"]][df.media_type == "not a book"].head()

Se verifican ahora los que se consideran *periodical*:

In [ ]:
df[["title", "media_type"]][df.media_type == "periodical"].head()

Por último, se revisa cuales son considerados nulos:

In [ ]:
df[["title", "media_type"]][df.media_type.isna()].head()

Se reemplazan los libros con **media_type** *NaN* por *Unknown*:

In [ ]:
df.media_type.fillna(value='unknown', inplace=True)
df.media_type.unique()

La columna **num_pages** contiene valores nulos. 
Hay que tenerlo en cuenta a la hora de calcular datos con ésta columna (p.ej la media, etc).

Además, se comprubea si hay libros con 0 páginas, en ese caso se modifica su valor por np.nan

In [ ]:
import numpy as np
df.loc[df.num_pages == 0, "num_pages"] = np.nan
df.num_pages.isna().sum()

In [ ]:
df.num_pages.loc[~df.num_pages.isna()].mean() #num_pages_mean sin tener en cuenta datos nulos

In [ ]:
df.num_pages.loc[~df.num_pages.isna()].astype("int64").unique()

Se analiza la columna **format** y se observan valores nulos

In [ ]:
df.format.unique()

Para evitar el borrado de muchos libros de forma inncesaria a todos los libros que no tengan un formato especificado se les asigna el formato "unknown"

In [ ]:
df.format = df.format.fillna(value="unknown")
df.format.value_counts()

Del mismo modo que la columna anterior, se analiza la columna **edition_information** y se tratan los valores nulos. 

In [ ]:
df.edition_information.head()

Para evitar el borrado de muchos libros de forma inncesaria se asigna el formato "unknown". 

In [ ]:
df.edition_information = df.edition_information.fillna(value="unknown")
df.edition_information.value_counts().head()

Se puede observar que en la mayoría de los casos no se ha propocionado información por lo que esta información puede terminar no siendo de utilidad. 

Sobre el campo **genres** se puede ver que es un campo con multivalores, por lo que el tratamiento que se hará en convertir el campo de generos a columnas, para ello se utiliza la función "get_dummies" que genera tantas columnas como generos haya.

Al ser una operación pesada, se vuelca el resultado en un csv y en posteriores ejecuciones del notebook se tiene en cuenta para leer del csv en lugar de generar los dummies.

In [ ]:
#df_genres_dummies = df.genres.str.get_dummies(sep=",")
#df_genres_dummies.to_csv("genres.csv")
df_genres_dummies = pd.read_csv("genres.csv")
df = df.join(df_genres_dummies)
df.columns

### Información sobre los usuarios <a class="anchor" name="columnas-usuarios"/>

Las columnas **books_count** y **best_book_id** no tienen un valor relevante para este estudio por lo que se eliminan del set de datos.

In [ ]:
df.drop(columns=["books_count", "best_book_id"], inplace=True)

Como se ha observado anteriormente, se ve si existen nulos en las columnas referidas a las valoraciones de los usuarios:

In [ ]:
print("reviews_count: {}".format(df.reviews_count.isna().sum()))
print("ratings_sum: {}".format(df.ratings_sum.isna().sum()))
print("ratings_count: {}".format(df.ratings_count.isna().sum()))
print("text_reviews_count: {}".format(df.text_reviews_count.isna().sum()))
print("text_reviews_count_global: {}".format(df.text_reviews_count_global.isna().sum()))
print("num_ratings_5: {}".format(df.num_ratings_5.isna().sum()))
print("num_ratings_4: {}".format(df.num_ratings_4.isna().sum()))
print("num_ratings_3: {}".format(df.num_ratings_3.isna().sum()))
print("num_ratings_2: {}".format(df.num_ratings_2.isna().sum()))
print("num_ratings_1: {}".format(df.num_ratings_1.isna().sum()))
print("average_rating: {}".format(df.average_rating.isna().sum()))
print("rating_count_global: {}".format(df.ratings_count_global.isna().sum()))
print("to read: {}".format(df.to_read.isna().sum()))
print("read: {}".format(df.read.isna().sum()))
print("currently reading: {}".format(df.currently_reading.isna().sum()))

Se analiza la columna **average_rating**. En caso de no tener valores, se deberían filtrar los registros que correspondan en una vista para hacer calculos. P.ej calcular los libros que tengan valoración por encima de la media.

A priori, no se observan libross sin valoración media.

In [ ]:
df.average_rating.fillna(value="unknow", inplace=True)
df.average_rating.value_counts().head()

A continuación se trata el campo **ratings_count_global**. Representa el total de valoraciones que ha recibido el libro.
En caso de tener algun valor nulo, se reemplaza por cero.

In [ ]:
df.ratings_count_global.fillna(value=0, inplace=True)

Además, el tipo de datos es el correcto para un contador

In [ ]:
df.ratings_count_global.dtype

Las columnas **to_read**, **read**, **currently_reading** se interprentan como un contador con la gente que ha leído, está leyendo o va leer el libro, por lo que se establece que los valores nulos se sustituyen por 0.

In [ ]:
df.to_read.fillna(value=0, inplace=True)
df.read.fillna(value=0, inplace=True)
df.currently_reading.fillna(value=0, inplace=True)

Dado que representa un número de personas, se expresan los resultados como números enteros

In [ ]:
df.to_read.astype('int64')
df.read.astype('int64')
df.currently_reading.astype('int64')

In [ ]:
df.info()

### Información sobre los autores <a class="anchor" name="columnas-autores"/>

A continuación se tratan las columnas que tiene información sobre los autores, ilustradores, etc.

In [ ]:
print("authors: {}".format(df.authors.isna().sum()))
print("illustrator: {}".format(df.illustrator.isna().sum()))
print("contributor: {}".format(df.contributor.isna().sum()))
print("editor: {}".format(df.editor.isna().sum()))
print("translator: {}".format(df.translator.isna().sum()))
print("narrator: {}".format(df.narrator.isna().sum()))


Para este caso, unicamente se van a tratar a los autores, puesto que el resto de campos tienen muchos valores nulos y no aportan mucho valor al análisis que se va a realizar.
Los autores nulos vamos a tratarlos como libros anonimos.

In [ ]:
df.authors.fillna(value="anonymous", inplace=True)
df.authors.isna().sum()

Para poder hacer operaciones con los autores, habría que realizar el mismo tratamiento que se ha hecho con los géneros. Al ser una operación muy pesada y no tener mucho valor para el análisis que se va a realizar a continuación, se prescinde de esta operación pero se deja en un notebook adicional (<a href="create_authors.ipynb">create_authors.ipynb</a>) que habría que hacer para llevar a cabo esta operación.

In [ ]:
#df.join(df.genres.str.get_dummies(sep=","))
#df_authors = pd.read_csv("autores.csv")
#df = df.join(df_authors)
#df.columns

## Distribución y Correlaciones <a class="anchor" name="distribucion-correlacion"/>

Tras la limpieza de datos nulos, se muestra como ha quedado la tabla

In [ ]:
df.head()

Se obtiene el año de cada libro a partir del campo **original_publication_date** mapeando el campo a datetime, eliminando los nulos que tiene un formato erróneo y aplicando una lambda que extrae el campo año y se añade como una nueva columna **publication_year**

In [ ]:
years = pd.to_datetime(df.original_publication_date, errors="coerce")
years.dropna(inplace=True)
years = years.apply(lambda x: int(x.year))
df['publication_year'] = years
is_older = (df.publication_year < 2008) & (df.publication_year > 1948) 
df = df[is_older]

Se eliminan los libros que no tienen número de páginas

In [ ]:
df.dropna(subset=["num_pages"], inplace=True)

df.isna().sum()

Se eliminan libros duplicados , esto es que compartan el campo **original_title**

In [ ]:
df.drop_duplicates(subset=["original_title"], inplace=True)

In [ ]:
# Cargar el CSV con los super-géneros

df_generos = pd.read_csv(filepath_or_buffer="./generos.csv")
df_generos

In [ ]:
# Un ejemplo si estuviesemos buscando el sub-género "action"
def get_super_genre(g):
    bool_df = df_generos.isin([g])
    bool_supergenre = bool_df.any()
    supergenre_index, = np.where(bool_supergenre == True)
    return bool_supergenre[supergenre_index[0]: supergenre_index[0] + 1].index[0]

# Agrupa la información de un genero por año.
# Recibe el género a buscar y el número total de años en el dataframe
# - En primer lugar filtra el dataframe recibido por género
# - Rellena los años perdidos
# - Aplica la funcion f al set de datos agrupados
def get_grouped_for_genre(df, g, max_year):
    global_dict = {}
    # Filtra el dataframe por genero
    df_g = df[g] == 1
    subdataframe = df[df_g][["publication_year", "average_rating", "ratings_count", "num_pages"]]
    # Introduce los años que no ha habido publicaciones para no dejar huecos
    min_year = subdataframe.publication_year.min().astype(dtype=np.int32)
    year_dict = []
    for year in range(min_year, max_year):
        year_dict.append({"publication_year" : year, "genre": g})
    year_df = pd.DataFrame(year_dict)
    year_df.year = year_df.publication_year.astype(dtype=np.int32)
    subdataframe = pd.merge(subdataframe, year_df, on="publication_year", how = "outer")
    # Agrupa el dataframe por año aplicando la funcion f
    grouped_by_year_genre = subdataframe.groupby("publication_year")
    grouped_by_year_genre = grouped_by_year_genre.apply(lambda x: f(x, g, global_dict))
    grouped_by_year_genre.reset_index(level=0, inplace=True)
    return grouped_by_year_genre

In [ ]:
# Se crea una funcion que se aplica a todas las agrupaciones por año.
# Esta función recibe, la agrupación, el género y un diccionario para ir agregando datos de años anteriores
# Como puede haber años donde no haya publicaciones del libro, se comprueba si no hay páginas ese año.
# - Si hay páginas se calcula el nuevo valor a partir del anterior
# - Si no hay páginas se propaga el valor anterior
# - Si es la primera ocurrencia se inicializa el diccionario
def f(x, g, global_dict):
    if g in global_dict:
        number_of_books = global_dict[g]["number_of_books"]
        average_rating = global_dict[g]["average_rating"]
        ratings_count = global_dict[g]["ratings_count"]
        num_pages = global_dict[g]["num_pages"]
        # puede que la serie agrupada por año no tenga valores 
        if x.num_pages.sum() > 0:
            number_of_books += x.average_rating.size
            average_rating = global_dict[g]["average_rating"] + (
                (x.average_rating.mean() - global_dict[g]["average_rating"]) / number_of_books)
            ratings_count += x.ratings_count.sum() 
            num_pages += x.num_pages.sum()
        
        global_dict[g] = {
            "average_rating" : average_rating, 
            "ratings_count" : ratings_count, 
            "num_pages": num_pages,
            "number_of_books": number_of_books,
            "genre" : g,
            "supergenre": get_super_genre(g)
        }
    else:
        global_dict[g] = {
            "average_rating" : x.average_rating.mean(), 
            "ratings_count" : x.ratings_count.sum(), 
            "num_pages": x.num_pages.sum(),
            "number_of_books": x.average_rating.size,
            "genre" : g,
            "supergenre": get_super_genre(g)
        }
    return pd.Series(global_dict[g])

    

#genres_columns = df.columns[37:214]
# removed genres "adult", "fiction", "non-fiction", "literature", "history", 
genres_columns = ["academic", "action", "aliens", "american", "animals", "anthropology", 
                  "adventure", "adult-fiction", "angels", "american-history", "nature", "art", 
                  "chick-lit", "dark", "demons", "americana", "movies", "business", "class", 
                  "death", "epic", "criticism", "picture-books", "cookbooks", "classic-literature", 
                  "erotica", "epic-fantasy", "christianity", "music", "cooking", "classics", "horror", 
                  "fairy-tales", "christian", "plays", "cultural", "christmas", "menage", "fantasy", 
                  "christian-fiction", "film", "economics", "comedy", "abuse", "ghosts", "gay", 
                  "education", "contemporary", "bdsm", "high-fantasy", "marriage", "faith", 
                  "cozy-mystery", "magic", "lgbt", "family", "crime", "magical-realism",
                  "parenting", "feminism", "detective", "mythology", "queer", "food", "drama", 
                  "paranormal", "relationships", "health", "post-apocalyptic", 
                  "society", "funny", "science-fiction", "spirituality", "language", 
                  "historical", "science-fiction-fantasy", "inspirational", "leadership", 
                  "historical-fiction", "shapeshifters", "humor", "steampunk", 
                  "mental-health", "literary-fiction", "speculative-fiction", "mental-illness", 
                  "love", "superheroes", "personal-development", "love-story", "supernatural", 
                  "philosophy", "medieval", "time-travel", "politics", "modern", "urban-fantasy", 
                  "psychology", "modern-classics", "vampires", "reference", 
                  "werewolves", "religion", "mystery", "witches", "research", "mystery-thriller", 
                  "dystopia", "science", "social", "realistic-fiction", "self-help", 
                  "romance", "social-issues", "suspense", "sociology", "thriller", "sports", 
                  "true-crime", "survival", "tragedy", "teaching", "war", "technology", "gothic", 
                  "theology", "travel", "writing", "medical"]
df_books_by_genre_and_year = pd.DataFrame()
max_year = df.publication_year.max().astype(dtype=np.int32)
#genres_columns
for g in genres_columns:
    grouped_by_year_genre = get_grouped_for_genre(df, g, max_year)
    df_books_by_genre_and_year = df_books_by_genre_and_year.append(grouped_by_year_genre)

df_books_by_genre_and_year.to_csv('books_by_genre_and_year.csv')

df_books_by_genre_and_year.shape

### Ir al data set graficos
<a href="./graficos.ipynb">Graficos</a>